In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import itertools
import math

import datetime

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.model_selection import train_test_split

import pandas as pd, numpy as np, time
from binance.client import Client
from binance.exceptions import BinanceAPIException

from IPython.display import clear_output

In [ ]:
trading_pair = "BTCUSDT"
api_key = ""
api_secret = ""

In [3]:
def connect_realnet():
    # Connect client to realnet api
    while True:
        try:
            client = Client(api_key, api_secret, testnet=False)
            break
        except Exception as e:
            print(e)
            time.sleep(10)
        
    client.API_URL = 'https://api.binance.com/api'
    return client

def get_kline(interval):
    import zipfile
    from urllib.request import urlopen
    import shutil
    import urllib
    import os
    import datetime
    from datetime import datetime
    from dateutil import rrule
    from os.path import exists

    colnames = ['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 
     'Close time', 'Quote asset volume', 'Number of trades', 
     'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']

    li = []
    start = datetime(2017, 8, 1)
    end = datetime.today()
    for dt in rrule.rrule(rrule.MONTHLY, dtstart=start, until=end.replace(month=end.month-1)):
        month = dt.strftime("%Y-%m")

    #     print(month)

        file_core = 'BTCUSDT-' + str(interval) + '-' + str(month)
        file_name = file_core + '.zip'
        file_path = "./klines/"

        if not exists(str(file_path)+str(file_core)+'.csv'):

            url = 'https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/' + str(interval) + '/' + file_name
            
#             print(url)

            # extracting zipfile from URL
            with urlopen(url) as response, open(file_path+file_name, 'wb') as out_file:
                shutil.copyfileobj(response, out_file)

                # extracting required file from zipfile
                try:
                    with zipfile.ZipFile(file_path+file_name) as zf:
                        zf.extract(str(file_core)+'.csv')
                        shutil.move(str(file_core)+'.csv', str(file_path)+str(file_core)+'.csv')
                        # deleting the zipfile from the directory
                        os.remove(file_path+file_name)
                        # loading data from the file
                        df = pd.read_csv(str(file_path)+str(file_core)+'.csv', names=colnames)
                        li.append(df)
                except Exception as e:
                    print(e)
        df = pd.read_csv(str(file_path)+str(file_core)+'.csv', names=colnames)
        li.append(df)
    thismonth = end.replace(day=1)
    client_r = connect_realnet()

    # request historical candle (or klines) data
    bars = []
    while True:
        try:
            t = int(time.time())
            bars = client_r.get_historical_klines(trading_pair, str(interval), str(thismonth.timestamp()))
            break
        except Exception as e:
            print(e)
            print('Error occured at %s.' % (datetime.datetime.now()))
            time.sleep(5)
            client_r = connect_realnet()
    bdf = pd.DataFrame(bars, columns=colnames)
    li.append(bdf)

    data = pd.concat(li, axis=0, ignore_index=True)
    return data.astype(float)

def update_kline(interval, df):
    
    client_r = connect_realnet()
    
    colnames = ['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 
     'Close time', 'Quote asset volume', 'Number of trades', 
     'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']
    
    # request historical candle (or klines) data
    bars = []
    while True:
        try:
            t = int(time.time())
            bars = client_r.get_historical_klines(trading_pair, interval, int(df['Close time'][len(df)-1]))
            break
        except Exception as e:
            print(e)
            print('Error occured at %s.' % (datetime.datetime.now()))
            time.sleep(1)
            client_r = connect_realnet()

    # put in dataframe
    bdf = pd.DataFrame(bars[:-1], columns=colnames)
    bdf = bdf.astype(float)
    
    df = df.append(bdf).reset_index(drop=True)
    
    return df

In [4]:
def truncate(number, decimals=0):
    """
    Returns a value truncated to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer.")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more.")
    elif decimals == 0:
        return math.trunc(number)

    factor = 10.0 ** decimals
    return math.trunc(number * factor) / factor

#buy a certain amount BTC
#Now with TAXES YEAH
def buy_t(acc_usdt, acc_btc, price, buy_amount):
    acc_usdt -= (buy_amount)*price
    acc_btc += buy_amount
    return acc_usdt, acc_btc

#sell a certain amount BTC
#Now with TAXES YEAH
def sell_t(acc_usdt, acc_btc, price, sell_amount):
    acc_usdt += (sell_amount)*price
    acc_btc -= sell_amount
    return acc_usdt, acc_btc

#buy an amount of BTC with USDT
def buy_b(acc_usdt, acc_btc, price, buy_amount):
    client_r = connect_realnet()
    try:
        return client_r.create_order(symbol = trading_pair, side='BUY', type='MARKET', quantity = buy_amount)
    except BinanceAPIException as e:
        print(e)
        return {'status':'FAILED'}

#sell an amount of BTC with USDT
def sell_b(acc_usdt, acc_btc, price, sell_amount):
    client_r = connect_realnet()
    try:
        return client_r.create_order(symbol = trading_pair, side='SELL', type='MARKET', quantity = sell_amount)
    except BinanceAPIException as e:
        print(e)
        return {'status':'FAILED'}

def buy(price, acc_usdt, acc_btc):
#     global acc_usdt
#     global acc_btc
    
    out = "Trying to buy..."
    quantity = truncate((acc_usdt/price)*0.995, 3)
#     quantity = ((int((acc_usdt/price)*10**6)-1)/10**6)
    if quantity > 0.0001:
        if(acc_usdt >= quantity*price):
            if real:
                buy_b(acc_usdt, acc_btc, price, quantity)
            else:
                acc_usdt, acc_btc = buy_t(acc_usdt, acc_btc, price, quantity)
            out = out + "Bought " + str(quantity) + "BTC at " + str(price)
            trade = True
            trade_type = "BUY"
        else:
            out = out + "No enough funds : " + str(acc_usdt) + " < " + str(buy_amount*price)
            trade = False
            trade_type = "NONE"
            print("BUY ERROR")
            print(out)
    else:
        out = out + "Trade too small : " + str(quantity)
        trade = False
        trade_type = "NONE"
#     print(out)
    return out, trade_type, trade, acc_usdt, acc_btc

def sell(price, acc_usdt, acc_btc):
#     global acc_usdt
#     global acc_btc
    
    out = "Trying to sell..."
    quantity = truncate(acc_btc, 3)
#     quantity = ((int(acc_btc*10**6)-1)/10**6)
    if quantity > 0.0001:
        if(acc_btc >= (quantity * 0.9999)):
            if real:
                sell_b(acc_usdt, acc_btc, price, quantity)
            else:
                acc_usdt, acc_btc = sell_t(acc_usdt, acc_btc, price, quantity)
            out = out + "Sold " + str(quantity) + "BTC at " + str(price)
            trade = True
            trade_type = "SELL"
        else:
            out = out + "No enough funds : " + str(acc_btc) + " < " + str(quantity)
            trade = False
            trade_type = "NONE"
            print("SELL ERROR")
            print(out)
    else:
        out = out + "Trade too small : " + str(quantity)
        trade = False
        trade_type = "NONE"
#     print(out)
    return out, trade_type, trade, acc_usdt, acc_btc

In [5]:
def tri(df):
    return pd.DataFrame(np.maximum((df['High'].values - df['Low'].values), 
                  (df['High'].values - df['Close'].values), 
                  (df['Low'].values - np.roll(df['Close'].values, 1, axis=0))))

def trip(df, p):
    return tri(df).rolling(p).sum()

def upmove(df):
    return (df['High'] - df['High'].shift())

def downmove(df):
    return (df['Low'].shift() - df['Low'])

def dmplus(df):
    return pd.DataFrame(np.maximum(upmove(df), 0))

def dmminus(df):
    return pd.DataFrame(np.maximum(downmove(df), 0))

def dmpplus(df, p):
    return dmplus(df).rolling(p).sum()

def dmpminus(df, p):
    return dmminus(df).rolling(p).sum()

def diplus(df, p):
    return np.divide(dmpplus(df, p).values, trip(df, p).values).flatten()

def diminus(df, p):
    return np.divide(dmpminus(df, p).values, trip(df, p).values).flatten()

def dx(df, cdf, p):
    dp = diplus(cdf, p)
    dm = diminus(cdf, p)
    dmdf = dp - dm
    dpdf = dp + dm
    DX = abs(dmdf)/dpdf
    df['DX'] = DX
    df['dp'] = dp
    df['dm'] = dm
    return df

def ma(df, w):
    test = df.rolling(window=w).mean()
    test = test.fillna(df)
    return test

In [6]:
def slow(bdf, a, b, c, d, size):
    bdf['Price Change'] = ((bdf['Close'] - bdf['Close'].shift())/bdf['Close'].shift()) + 1
    bdf['Action'] = 'NONE'
    bdf['Current Action'] = 'SELL'
    bdf['Want'] = 'NONE'
    bdf['Current Want'] = 'NONE'
    bdf['Move'] = 0
    bdf['Account Value'] = 1
    
    tdf = pd.DataFrame()
    tdf = dx(tdf, bdf, size)
    bdf['DX'], bdf['dp'], bdf['dm'] = tdf['DX'], tdf['dp'], tdf['dm']
    
    for i in range(size+1, len(bdf)):
        
        if bdf['dp'][i-1] > bdf['dm'][i-1]:
            bdf['Move'][i] = bdf['dp'][i-1]
        else:
            bdf['Move'][i] = -bdf['dm'][i-1]
    
        if bdf['DX'][i-1] > a and bdf['Move'][i] > c:
            bdf['Want'][i] = 'SELL'
        if bdf['DX'][i-1] > b and bdf['Move'][i] < d:
            bdf['Want'][i] = 'BUY'
            
        if bdf['Want'][i] != 'NONE' and bdf['Want'][i] != bdf['Current Want'][i-1]:
            bdf['Current Want'][i] = bdf['Want'][i]
        else:
            bdf['Current Want'][i] = bdf['Current Want'][i-1]
            bdf['Want'][i] = 'NONE'
        
        if (bdf['Current Want'][i] != bdf['Current Want'][i-1]):
            bdf['Action'][i] = bdf['Current Want'][i]
    
        if bdf['Action'][i] != 'NONE' and bdf['Action'][i] != bdf['Current Action'][i-1]:
            bdf['Current Action'][i] = bdf['Action'][i]
        else:
            bdf['Current Action'][i] = bdf['Current Action'][i-1]
            bdf['Action'][i] = 'NONE'

    bdf['Account Change'] = 1
    bdf['Account Value'] = 1
    bdf.loc[bdf['Current Action'] == 'BUY' , 'Account Change'] = bdf['Price Change']
    bdf.loc[bdf['Current Action'] == 'SELL', 'Account Change'] = 1
    bdf['Account Value'] = bdf['Account Change'].cumprod()*bdf['Account Value']
    print(bdf['Account Value'][len(bdf)-1])
    return bdf['Account Value'][len(bdf)-1]

In [7]:
def fast(bdf, a, b, c, d, size):
    bdf['Price Change'] = ((bdf['Close'] - bdf['Close'].shift())/bdf['Close'].shift()) + 1
    bdf['Action'] = 'NONE'
    bdf['Current Action'] = 'SELL'
    bdf['Want'] = 'NONE'
    bdf['Current Want'] = 'NONE'
    bdf['Move'] = 0
    bdf['Account Value'] = 1
    
    tdf = pd.DataFrame()
    tdf = dx(tdf, bdf, size).shift()
    bdf['DX'], bdf['dp'], bdf['dm'] = tdf['DX'], tdf['dp'], tdf['dm']
    
    bdf['Move'] = bdf['dp']
    bdf.loc[bdf['dm'] > bdf['dp'], 'Move'] = -bdf['dm']
    bdf['Want'] = 'NONE'
    bdf.loc[(bdf['DX'] > a) & (bdf['Move'] > c), 'Want'] = 'SELL'
    bdf.loc[(bdf['DX'] > b) & (bdf['Move'] < d), 'Want'] = 'BUY'
    
    bdf['Current Want'] = bdf['Want']
    bdf.loc[bdf['Current Want'] == 'NONE', 'Current Want'] = None
    bdf['Current Want'] = bdf['Current Want'].fillna(method='ffill')
    
    bdf.loc[(bdf['Current Want'] != bdf['Current Want'].shift()), 'Action'] = bdf['Current Want']
    
    bdf['Current Action'] = bdf['Action']
    bdf.loc[bdf['Current Action'] == 'NONE', 'Current Action'] = None
    bdf['Current Action'] = bdf['Current Action'].fillna(method='ffill')

    bdf['Dup'] = (bdf['Current Action'] == bdf['Current Action'].shift())
    bdf['Action'] = bdf['Current Action']*(1 - bdf['Dup'])
    
    bdf = bdf.drop(columns=['Dup'])
        
    bdf['Account Change'] = 1
    bdf['Account Value'] = 1
    bdf.loc[bdf['Current Action'] == 'BUY' , 'Account Change'] = bdf['Price Change']
    bdf.loc[bdf['Current Action'] == 'SELL', 'Account Change'] = 1
    bdf['Account Value'] = bdf['Account Change'].cumprod()*bdf['Account Value']
    return bdf, bdf['Account Value'][len(bdf)-1]

In [8]:
def fast_2(bdf, al, bl, cl, dl, sizel, a, b, c, d, size):
    bdf['Price Change'] = ((bdf['Open'] - bdf['Open'].shift())/bdf['Open'].shift()) + 1
    bdf['Action'] = 'NONE'
    bdf['Current Action'] = 'SELL'
    bdf['Want'] = 'NONE'
    bdf['Current Want'] = 'NONE'
    bdf['Move'] = 0
    bdf['Account Value'] = 1
    
    ldf = pd.DataFrame()
    ldf = dx(ldf, bdf, sizel).shift()
    bdf['DXl'], bdf['dpl'], bdf['dml'] = ldf['DX'], ldf['dp'], ldf['dm']
    
    sdf = pd.DataFrame()
    sdf = dx(sdf, bdf, size).shift()
    bdf['DX'], bdf['dp'], bdf['dm'] = sdf['DX'], sdf['dp'], sdf['dm']
    
    bdf['Movel'] = bdf['dpl']
    bdf.loc[bdf['dml'] > bdf['dpl'], 'Movel'] = -bdf['dml']
    bdf['Move'] = bdf['dp']
    bdf.loc[bdf['dm'] > bdf['dp'], 'Move'] = -bdf['dm']
    #--------------------
    bdf['Wantl'] = 'NONE'
    bdf.loc[(bdf['DXl'] > al) & (bdf['Movel'] > cl), 'Wantl'] = 'SELL'
    bdf.loc[(bdf['DXl'] > bl) & (bdf['Movel'] < dl), 'Wantl'] = 'BUY'
    
    bdf['Current Wantl'] = bdf['Wantl']
    bdf.loc[bdf['Current Wantl'] == 'NONE', 'Current Wantl'] = None
    bdf['Current Wantl'] = bdf['Current Wantl'].fillna(method='ffill')
    #--------------------
    bdf['Want'] = 'NONE'
    bdf.loc[(bdf['DX'] > a) & (bdf['Move'] > c), 'Want'] = 'SELL'
    bdf.loc[(bdf['DX'] > b) & (bdf['Move'] < d), 'Want'] = 'BUY'
    
    bdf['Current Want'] = bdf['Want']
    bdf.loc[bdf['Current Want'] == 'NONE', 'Current Want'] = None
    bdf['Current Want'] = bdf['Current Want'].fillna(method='ffill')    
    #--------------------
    bdf.loc[(bdf['Current Wantl'] != bdf['Current Want']), 'Action'] = bdf['Current Want']
    
    bdf['Current Action'] = bdf['Action']
    bdf.loc[bdf['Current Action'] == 'NONE', 'Current Action'] = None
    bdf['Current Action'] = bdf['Current Action'].fillna(method='ffill')
    #--------------------
    bdf['Dup'] = (bdf['Current Action'] == bdf['Current Action'].shift())
    bdf['Action'] = bdf['Current Action']*(1 - bdf['Dup'])
    
    bdf = bdf.drop(columns=['Dup'])
        
    bdf['Account Change'] = 1
    bdf['Account Value'] = 1
    bdf.loc[bdf['Current Action'] == 'BUY' , 'Account Change'] = bdf['Price Change']
    bdf.loc[bdf['Current Action'] == 'SELL', 'Account Change'] = 1
    bdf['Account Value'] = bdf['Account Change'].cumprod()*bdf['Account Value']
    return bdf, bdf['Account Value'][len(bdf)-1]

In [9]:
df = get_kline('1m')
len(df)

2665240

In [10]:
bdf = df[len(df)-100:].reset_index(drop=True)

In [11]:
file = "v2_log.csv"
cycle = 60
real = True
params = [0.74, 0.79, 0.85, 0.17, 99, 0.01, 0.11, 0.15, 0.03, 3]

In [12]:
%matplotlib inline

f = fast_2
bdf, value = f(bdf, *params)
bdf['True Account'] = 0
prev_time = 0

while True:
    t = int(time.time())
    if t % cycle == 1 and t != prev_time:
        
        prev_time = t
        
        now = datetime.datetime.now()
        print(now.strftime("%Y-%m-%d %H:%M:%S"))
        
        bdf = update_kline("1m", bdf)
        
        bdf, value = f(bdf, *params)
        
        client_r = connect_realnet()
        
        while True:
            try:
                acc_usdt = float(client_r.get_asset_balance(asset='USDT')['free'])
                acc_btc = float(client_r.get_asset_balance(asset='BTC')['free'])
                break
            except Exception as e:
                print("Failed to read states from server during init : ")
                print(e)
                client_r = connect_realnet()
        
        price = bdf['Close'][len(bdf)-1]
        
        if bdf['Action'][len(bdf)-1] == 'BUY':
            out, trade_type, trade, acc_usdt, acc_btc = buy(price, acc_usdt, acc_btc)
            if not(trade):
                bdf['Action'][len(bdf)-1] = None
        elif bdf['Action'][len(bdf)-1] == 'SELL':
            out, trade_type, trade, acc_usdt, acc_btc = sell(price, acc_usdt, acc_btc)
            if not(trade):
                bdf['Action'][len(bdf)-1] = None
        else:
            out = "Nothing to do"
        print(out)
        
        while True:
            try:
                acc_usdt = float(client_r.get_asset_balance(asset='USDT')['free'])
                acc_btc = float(client_r.get_asset_balance(asset='BTC')['free'])
                break
            except Exception as e:
                print("Failed to read states from server during init : ")
                print(e)
                client_r = connect_realnet()
        
        bdf['True Account'][len(bdf)-1] = price*acc_btc + acc_usdt
        print("Account :", price*acc_btc + acc_usdt, " USDT")
        
        bdf.to_csv(file)

2022-09-17 19:17:01
Nothing to do
Account : 914.0072184516999  USDT
2022-09-17 19:18:01
Nothing to do
Account : 914.0081041034999  USDT
2022-09-17 19:19:01
Nothing to do
Account : 913.6959118439999  USDT
2022-09-17 19:20:01
Nothing to do
Account : 913.4572286838999  USDT


KeyboardInterrupt: 